In [1]:
import nltk
import numpy as np
#nltk.download('punkt')
from nltk.stem.porter import PorterStemmer
from torch.utils.data import Dataset,DataLoader


stemmer=PorterStemmer()

def tokenize(sentence):
    return nltk.word_tokenize(sentence)

def stem(word):
    return stemmer.stem(word.lower())


def bag_of_words(tokenized_sentence,all_words):
    tokenized_sentence=[stem(w) for w in tokenized_sentence]
    bag=np.zeros(len(all_words),dtype=np.float32)

    for idx,w in enumerate(all_words):
        if  w in tokenized_sentence:
            bag[idx]=1.0
    return bag


a=['a','q','q','w']    
words=['a','b','q','r','t','w']
bag=bag_of_words(a,words)
print(bag)

[1. 0. 1. 0. 0. 1.]


In [2]:
import torch
import torch.nn as nn
 

class NeuralNet(nn.Module):
    def __init__(self,input_size,hidden_size,num_classes):
         super(NeuralNet,self).__init__()
         self.l1=nn.Linear(input_size,hidden_size)
         self.l2=nn.Linear(hidden_size,hidden_size)
         self.l3=nn.Linear(hidden_size,num_classes)
         self.relu=nn.ReLU()

    def forward(self,x):
        output=self.relu(self.l1(x))
        output=self.relu(self.l2(output))
        output=self.l3(output)     
        return output

In [3]:
class ChatDataset(Dataset):

    def __init__(self):
        self.n_samples = len(X_train)
        self.x_data = X_train
        self.y_data = y_train

    # support indexing such that dataset[i] can be used to get i-th sample
    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    # we can call len(dataset) to return the size
    def __len__(self):
        return self.n_samples

In [4]:
import numpy as np
import random
import json

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

#from nltk_utils import bag_of_words, tokenize, stem
#from model import NeuralNet

with open('../input/chtabot/intents.json', 'r') as f:
    intents = json.load(f)

all_words = []
tags = []
xy = []
# loop through each sentence in our intents patterns
for intent in intents['intents']:
    tag = intent['tag']
    # add to tag list
    tags.append(tag)
    for pattern in intent['patterns']:
        # tokenize each word in the sentence
        w = tokenize(pattern)
        # add to our words list
        all_words.extend(w)
        # add to xy pair
        xy.append((w, tag))

# stem and lower each word
ignore_words = ['?', '.', '!']
all_words = [stem(w) for w in all_words if w not in ignore_words]
# remove duplicates and sort
all_words = sorted(set(all_words))
tags = sorted(set(tags))

print(len(xy), "patterns")
print(len(tags), "tags:", tags)
print(len(all_words), "unique stemmed words:", all_words)

# create training data
X_train = []
y_train = []
for (pattern_sentence, tag) in xy:
    # X: bag of words for each pattern_sentence
    bag = bag_of_words(pattern_sentence, all_words)
    X_train.append(bag)
    # y: PyTorch CrossEntropyLoss needs only class labels, not one-hot
    label = tags.index(tag)
    y_train.append(label)

X_train = np.array(X_train)
y_train = np.array(y_train)

60 patterns
20 tags: ['Company', 'Weather', 'contact', 'delivery', 'funny', 'goodbye', 'greeting', 'job', 'location', 'missing_id', 'noanswer', 'opentoday', 'options', 'order_components', 'order_tracking', 'payments', 'search_department', 'service', 'thanks', 'unknown']
97 unique stemmed words: ["'s", '345a23', '431b67', '561a24', '562b78', 'a', 'accept', 'ani', 'antopoli', 'anyon', 'are', 'assist', 'be', 'bye', 'can', 'card', 'cash', 'compani', 'compon', 'compris', 'contact', 'could', 'credit', 'date', 'day', 'deliveri', 'depart', 'do', 'doe', 'find', 'funni', 'get', 'good', 'goodby', 'have', 'hello', 'help', 'hey', 'hi', 'hour', 'how', 'i', 'id', 'in', 'internship', 'is', 'job', 'joke', 'kind', 'know', 'later', 'list', 'locat', 'long', 'lot', 'mastercard', 'me', 'my', 'need', 'of', 'offic', 'onli', 'open', 'order', 'pay', 'paypal', 'problem', 'provid', 'see', 'sell', 'servic', 'ship', 'someth', 'take', 'talk', 'tell', 'thank', 'that', 'the', 'there', 'thi', 'to', 'today', 'touch', 't

In [5]:
# Hyper-parameters 
num_epochs = 1000
batch_size = 8
learning_rate = 0.001
input_size = len(X_train[0])
hidden_size = 1024
output_size = len(tags)
print(input_size, output_size)


97 20


In [6]:
dataset = ChatDataset()
train_loader = DataLoader(dataset=dataset,
                          batch_size=batch_size,
                          shuffle=True,
                          num_workers=0)

In [7]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = NeuralNet(input_size, hidden_size, output_size).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)


In [8]:
# Train the model
for epoch in range(num_epochs):
    for (words, labels) in train_loader:
        words = words.to(device)
        labels = labels.to(dtype=torch.long).to(device)
        
        # Forward pass
        outputs = model(words)
        # if y would be one-hot, we must apply
        # labels = torch.max(labels, 1)[1]
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    if (epoch+1) % 100 == 0:
        print (f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')


print(f'final loss: {loss.item():.4f}')



Epoch [100/1000], Loss: 0.0001
Epoch [200/1000], Loss: 0.0000
Epoch [300/1000], Loss: 0.0000
Epoch [400/1000], Loss: 0.0000
Epoch [500/1000], Loss: 0.0000
Epoch [600/1000], Loss: 0.0000
Epoch [700/1000], Loss: 0.0000
Epoch [800/1000], Loss: 0.0000
Epoch [900/1000], Loss: 0.0000
Epoch [1000/1000], Loss: 0.0000
final loss: 0.0000


In [9]:
data = {
"model_state": model.state_dict(),
"input_size": input_size,
"hidden_size": hidden_size,
"output_size": output_size,
"all_words": all_words,
"tags": tags
}

FILE = "data.pth"
torch.save(data, FILE)

print(f'training complete. file saved to {FILE}')

training complete. file saved to data.pth


In [ ]:
import random
import json

import torch

#from model import NeuralNet
#from nltk_utils import bag_of_words, tokenize

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

with open('../input/chtabot/intents.json', 'r') as json_data:
    intents = json.load(json_data)

FILE = "data.pth"
data = torch.load(FILE)

input_size = data["input_size"]
hidden_size = data["hidden_size"]
output_size = data["output_size"]
all_words = data['all_words']
tags = data['tags']
model_state = data["model_state"]

model = NeuralNet(input_size, hidden_size, output_size).to(device)
model.load_state_dict(model_state)
model.eval()

bot_name = "Sam"
print("Let's chat! (type 'quit' to exit)")
while True:
    # sentence = "do you use credit cards?"
    sentence = input("You: ")
    if sentence == "quit":
        break

    sentence = tokenize(sentence)
    X = bag_of_words(sentence, all_words)
    X = X.reshape(1, X.shape[0])
    X = torch.from_numpy(X).to(device)

    output = model(X)
    _, predicted = torch.max(output, dim=1)

    tag = tags[predicted.item()]

    probs = torch.softmax(output, dim=1)
    prob = probs[0][predicted.item()]
    if prob.item() > 0.75:
        for intent in intents['intents']:
            if tag == intent["tag"]:
                print(f"{bot_name}: {random.choice(intent['responses'])}")
    else:
        print(f"{bot_name}: I do not understand...")

Let's chat! (type 'quit' to exit)


You:  who are you


Sam: Antopolis is an innovation driven enterprise that provides a range of tech and marketing services tailored to EMPOWER CHANGEMAKERS.


You:  you who


Sam: Antopolis is an innovation driven enterprise that provides a range of tech and marketing services tailored to EMPOWER CHANGEMAKERS.


You:  what type of work do you do for people


Sam: We provide a range of tech and marketing services
